In [2]:
from transformers import pipeline, DetrConfig, DetrForObjectDetection, DetrImageProcessor
from torchvision.transforms import ToTensor
from PIL import Image
import cv2
import numpy as np
import torch
import os


/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
pipeline = pipeline(
    "object-detection", 
    model="facebook/detr-resnet-50",
    dtype=torch.float16,
    device_map=0
)

pipeline("http://images.cocodataset.org/val2017/000000039769.jpg")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8474a9c7-366d-44ee-8584-94b1ee5ccd52)')' thrown while requesting HEAD https://huggingface.co/facebook/detr-resnet-50/resolve/main/config.json
Retrying in 1s [Retry 1/5].
/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in t

[{'score': 0.998046875,
  'label': 'remote',
  'box': {'xmin': 40, 'ymin': 70, 'xmax': 175, 'ymax': 118}},
 {'score': 0.99609375,
  'label': 'remote',
  'box': {'xmin': 333, 'ymin': 72, 'xmax': 368, 'ymax': 187}},
 {'score': 0.99560546875,
  'label': 'couch',
  'box': {'xmin': 0, 'ymin': 1, 'xmax': 639, 'ymax': 473}},
 {'score': 0.9990234375,
  'label': 'cat',
  'box': {'xmin': 13, 'ymin': 52, 'xmax': 314, 'ymax': 471}},
 {'score': 0.99853515625,
  'label': 'cat',
  'box': {'xmin': 345, 'ymin': 23, 'xmax': 640, 'ymax': 368}}]

In [5]:
model_name = "facebook/detr-resnet-50"
processor = DetrImageProcessor.from_pretrained(model_name)
model = DetrForObjectDetection.from_pretrained(model_name)

image_path = "../data/MOT20/train/MOT20-01/img1/000004.jpg"
pil_image = Image.open(image_path).convert("RGB")

inputs = processor(pil_image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

target_sizes = torch.tensor([pil_image.size[::-1]])

outputs = processor.post_process_object_detection(
    outputs,
    target_sizes=target_sizes
)[0]


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
outputs

{'scores': tensor([0.6665, 0.8918, 0.8335, 0.8576, 0.9848, 0.8956, 0.6130, 0.8695, 0.8047,
         0.9593, 0.5506, 0.8283, 0.8241, 0.7151, 0.9338, 0.6488, 0.7531, 0.9424,
         0.7225, 0.9828, 0.8286, 0.5618, 0.6591, 0.8986, 0.9903, 0.9651, 0.9369,
         0.9005, 0.8717, 0.9823, 0.7156, 0.6560, 0.9344, 0.5435, 0.6280]),
 'labels': tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]),
 'boxes': tensor([[1075.0922,  621.8323, 1121.7284,  640.5376],
         [ 939.1262,  418.1636, 1018.8958,  629.5999],
         [1048.9517,  621.9061, 1092.7832,  638.4801],
         [ 715.7448,  601.7747,  784.5789,  628.6303],
         [1417.5613,  636.5034, 1568.1194,  672.0249],
         [1688.2906,  236.6479, 1864.4816,  692.5023],
         [1556.1842,  624.2912, 1722.5892,  674.7914],
         [1893.6212,  598.3700, 2047.7789,  688.2537],
         [1859.0291,  663.0734, 2037.8545,  702.7540],
         [ 777.7662,  608.0922,  

In [6]:
# Convert PIL image to OpenCV format
cv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

# Draw boxes
for score, label, box in zip(outputs["scores"], outputs["labels"], outputs["boxes"]):
    if score < 0.7:  # confidence threshold
        continue
    box = [int(i) for i in box.tolist()]
    x0, y0, x1, y1 = box
    cv2.rectangle(cv_image, (x0, y0), (x1, y1), (0, 255, 0), 2)
    cv2.putText(
        cv_image,
        f"{model.config.id2label[label.item()]}: {score:.2f}",
        (x0, y0 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        1,
        cv2.LINE_AA,
    )

# Display or save
cv2.imwrite("detr_output.png", cv_image)



True

In [27]:
def detect_image_from_folder(folder_path, output_folder):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            pil_image = Image.open(image_path).convert("RGB")
            inputs = processor(pil_image, return_tensors="pt")

            with torch.no_grad():
                outputs = model(**inputs)

            target_sizes = torch.tensor([pil_image.size[::-1]])

            outputs = processor.post_process_object_detection(
                outputs,
                target_sizes=target_sizes
            )[0]

            cv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

            for score, label, box in zip(outputs["scores"], outputs["labels"], outputs["boxes"]):
                if score < 0.7:
                    continue
                box = [int(i) for i in box.tolist()]
                x0, y0, x1, y1 = box
                cv2.rectangle(cv_image, (x0, y0), (x1, y1), (0, 255, 0), 2)
                cv2.putText(
                    cv_image,
                    f"{model.config.id2label[label.item()]}: {score:.2f}",
                    (x0, y0 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    1,
                    cv2.LINE_AA,
                )

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, cv_image)

In [28]:
detect_image_from_folder("../data/pohang/", "experiments/detr/predictions/")